<a href="https://colab.research.google.com/github/akiran703/MultiInputPatientApproach/blob/master/baselinemodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow.keras as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.layers import Dense, Input,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten,Concatenate
from tensorflow.keras.utils import to_categorical
import os
import pandas as pd
import csv
import cv2
import random
import glob
from IPython.core.debugger import Tracer

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth 
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
download_CT_COVID = drive.CreateFile({'id':'1bDI0Lokr1Do1adDq6kY2I6NIedaPGDbS'})
download_CT_COVID.GetContentFile('clustercovid.zip')

In [ ]:
!unzip /content/clustercovid.zip -d /content/Data

Archive:  /content/clustercovid.zip
   creating: /content/Data/clustercovid/
  inflating: /content/Data/clustercovid/.DS_Store  
  inflating: /content/Data/__MACOSX/clustercovid/._.DS_Store  
   creating: /content/Data/clustercovid/coviddata/
  inflating: /content/Data/clustercovid/firstcovid.py  
  inflating: /content/Data/__MACOSX/clustercovid/._firstcovid.py  
   creating: /content/Data/clustercovid/coviddata/CT_COVID/
  inflating: /content/Data/__MACOSX/clustercovid/coviddata/._CT_COVID  
  inflating: /content/Data/clustercovid/coviddata/.DS_Store  
  inflating: /content/Data/__MACOSX/clustercovid/coviddata/._.DS_Store  
   creating: /content/Data/clustercovid/coviddata/CT_NonCOVID/
  inflating: /content/Data/__MACOSX/clustercovid/coviddata/._CT_NonCOVID  
  inflating: /content/Data/clustercovid/coviddata/CT_COVID/2020.03.01.20029769-p21-73_1%1.png  
  inflating: /content/Data/__MACOSX/clustercovid/coviddata/CT_COVID/._2020.03.01.20029769-p21-73_1%1.png  
  inflating: /content/Data

In [ ]:
download_files = drive.CreateFile({'id':'13VzXjmQ8LE65FtIdtz841w-JvpJ-SYU3'})
download_files.GetContentFile('COVID-CT-masteractual.zip')

In [ ]:
!unzip /content/COVID-CT-masteractual.zip -d /content/textfile

Archive:  /content/COVID-CT-masteractual.zip
0c83254a43230de176489a9b4e3ac12e23b0df53
   creating: /content/textfile/COVID-CT-master/
  inflating: /content/textfile/COVID-CT-master/COVID-CT-MetaInfo.xlsx  
   creating: /content/textfile/COVID-CT-master/Data-split/
   creating: /content/textfile/COVID-CT-master/Data-split/COVID/
  inflating: /content/textfile/COVID-CT-master/Data-split/COVID/testCT_COVID.txt  
  inflating: /content/textfile/COVID-CT-master/Data-split/COVID/trainCT_COVID.txt  
  inflating: /content/textfile/COVID-CT-master/Data-split/COVID/valCT_COVID.txt  
   creating: /content/textfile/COVID-CT-master/Data-split/NonCOVID/
  inflating: /content/textfile/COVID-CT-master/Data-split/NonCOVID/CT_NonCOVID_test_id.csv  
  inflating: /content/textfile/COVID-CT-master/Data-split/NonCOVID/CT_NonCOVID_train_id.csv  
  inflating: /content/textfile/COVID-CT-master/Data-split/NonCOVID/CT_NonCOVID_val_id.csv  
  inflating: /content/textfile/COVID-CT-master/Data-split/NonCOVID/testCT_

In [ ]:
img_dir = "/content/Data/clustercovid/coviddata/CT_COVID"
img_dirnon = "/content/Data/clustercovid/coviddata/CT_NonCOVID"

ct_dir = "/content/textfile/COVID-CT-master/Data-split/COVID/testCT_COVID.txt"
tr_dir = "/content/textfile/COVID-CT-master/Data-split/COVID/trainCT_COVID.txt"
vl_dir = "/content/textfile/COVID-CT-master/Data-split/COVID/valCT_COVID.txt"

nct_dir = "/content/textfile/COVID-CT-master/Data-split/NonCOVID/testCT_NonCOVID.txt"
ntr_dir = "/content/textfile/COVID-CT-master/Data-split/NonCOVID/trainCT_NonCOVID.txt"
nvl_dir = "/content/textfile/COVID-CT-master/Data-split/NonCOVID/valCT_NonCOVID.txt"

CATEGORIES = ["covid", "notcovid"]

In [ ]:
data_path = os.path.join(img_dir, '*g')
data_pathnon = os.path.join(img_dirnon, '*g')

files = glob.glob(data_path)
files = sorted(files, key=str.lower)
filesnon = glob.glob(data_pathnon)
filesnon = sorted(filesnon, key=str.lower)

covidTest = []
covidTestlabel = []

covidTrain = []
covidTrainlabel = []

covidValid = []
covidValidlabel = []

nontest = []
nontestlabel = []

nontrain = []
nontrainlabel = []

nonvalid = []
nonvalidlabel = []

ctFile = open(ct_dir, 'r')
ctLine = ctFile.readlines()

trFile = open(tr_dir, 'r')
tdLine = trFile.readlines()

vlFile = open(vl_dir, 'r')
vlLine = vlFile.readlines()

nonctfile = open(nct_dir, 'r')
nonctline = nonctfile.readlines()

nontrfile = open(ntr_dir, 'r')
nontrline = nontrfile.readlines()

nonvlfile = open(nvl_dir, 'r')
nonvlline = nonvlfile.readlines()

In [ ]:
# changed from 500 to 100 for debug
desired_size = 200

In [ ]:
for f1 in files:
    img = cv2.imread(f1)
    #imgreszie = cv2.resize(img,(480,480))
    old_size = img.shape[:2]
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    img = cv2.resize(img, (new_size[1], new_size[0]))
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top,bottom = delta_h//2, delta_h-(delta_h//2)
    left,right = delta_w//2, delta_w-(delta_w//2)
    color = [0,0,0]
    imgreszie = cv2.copyMakeBorder(img,top,bottom,left,right,cv2.BORDER_CONSTANT,value=color)
    normresize = cv2.normalize(imgreszie, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    f1 = f1[f1.rfind('COVID/') + 6:]
    for line in ctLine:
        if line.replace('\r\n', '').rfind(f1) == 0:
            covidTest.append((np.array(normresize)))
            covidTestlabel.append(1)

    for line in tdLine:
        if line.replace('\r\n', '').rfind(f1) == 0:
            covidTrain.append((np.array(normresize)))
            covidTrainlabel.append(1)


    for line in vlLine:
        if line.replace('\r\n', '').rfind(f1) == 0:
            covidValid.append((np.array(normresize)))
            covidValidlabel.append(1)

In [ ]:
for f1 in filesnon:
    img = cv2.imread(f1)
    #imgreszie = cv2.resize(img, (480, 480))
    old_size = img.shape[:2]
    ratio = float(desired_size) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    img = cv2.resize(img, (new_size[1], new_size[0]))
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)
    color = [0, 0, 0]
    imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    #print(np.max(imgreszie))
    #print(np.min(imgreszie))
    normresize = (imgreszie* 1.0)/np.max(imgreszie)
    #print(np.max(normresize))
    #print(np.min(normresize))

    #normresize = cv2.normalize(imgreszie,None,alpha=0,beta=1,norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_32F)
    origf1 = f1
    f1 = f1[f1.rfind('COVID/') + 6:]
    for line in nonctline:
        if line.replace('\r\n', '').rfind(f1) == 0:
            nontest.append(normresize)
            nontestlabel.append(0)

    for line in nontrline:
        if line.replace('\r\n', '').rfind(f1) == 0:
            nontrain.append(normresize)
            nontrainlabel.append(0)

    for line in nonvlline:
        if line.replace('\r\n', '').rfind(f1) == 0:
            nonvalid.append(normresize)
            nonvalidlabel.append(0)

In [ ]:
combinetraindata = covidTrain + nontrain
print(len(combinetraindata))
combinetestdata = covidTest + nontest
combinevaliddata = covidValid + nonvalid

combinetrainlabel = covidTrainlabel + nontrainlabel
combinetrainlabel = to_categorical(combinetrainlabel)
combinetestlabel = covidTestlabel + nontestlabel
combinetestlabel = to_categorical(combinetestlabel)
combinevalidlabel = covidValidlabel + nonvalidlabel
combinevalidlabel = to_categorical(combinevalidlabel)

# Randomize the data
rand_order = np.random.permutation(len(combinetraindata))
combinetraindata = np.array(combinetraindata)
combinetraindata = combinetraindata[rand_order, :, :, :]
combinetrainlabel = combinetrainlabel[rand_order]

rand_order = np.random.permutation(len(combinetestdata))
combinetestdata = np.array(combinetestdata)
combinetestdata = combinetestdata[rand_order, :, :, :]
combinetestlabel = combinetestlabel[rand_order]

rand_order = np.random.permutation(len(combinevaliddata))
combinevaliddata = np.array(combinevaliddata)
combinevaliddata = combinevaliddata[rand_order, :, :, :]
combinevalidlabel = combinevalidlabel[rand_order]


425


In [ ]:
inputs6 = Input(shape = (desired_size, desired_size, 3))
add16 = (Conv2D(96, (3, 3), activation='relu', padding='valid'))(inputs6)
max16 = (MaxPooling2D((2, 2)))(add16)
add26 = (Conv2D(256, (5, 5), activation='relu', strides=1, padding='same'))(max16)
max26 = (MaxPooling2D((2, 2)))(add26)
add36 = (Conv2D(384, (3, 3), activation='relu', strides=1, padding='same'))(max26)
add46 = (Conv2D(384, (3, 3), activation='relu', strides=1, padding='same'))(add36)
add56 = (Conv2D(256, (3, 3), activation='relu', strides=1, padding='same'))(add46)
max36 = (MaxPooling2D((2, 2)))(add56)



addfinal = (Conv2D(128,(3,3),activation ='relu',padding='valid'))(max36)
finalmax = (MaxPooling2D((2,2)))(addfinal)
flattenmax = Flatten()(finalmax)
dense1 = (Dense(256, activation='relu'))(flattenmax)
dense2 = (Dense(512, activation='relu'))(dense1)
output = (Dense(2, activation='softmax'))(dense2)
model = Model(inputs = [inputs6],outputs = output)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 98, 98, 96)        2688      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 49, 49, 96)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 49, 49, 256)       614656    
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 24, 24, 256)       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 24, 24, 384)       885120    
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 24, 24, 384)       1327

In [ ]:
#save weights
checkpoint_filepath = '/content/checkpoint_paper.hdf5'
model_checkpoint_callback = tf.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
csv_log_paper = CSVLogger("results_paper.csv")

In [ ]:
# complie
optimizer = tf.optimizers.SGD(learning_rate=0.05)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(combinetraindata, combinetrainlabel, batch_size=32, epochs=50, callbacks=[model_checkpoint_callback],verbose=True, validation_data=(combinevaliddata,combinevalidlabel))
#model.load_weights(checkpoint_filepath)
#loss, accuracy = model.evaluate(combinetestdata, combinetestlabel, verbose=True)
#print(f'Test loss: {loss:.3}')
#print(f'Test accuracy: {accuracy:.3}')

Epoch 1/50
14/14 [==============================] - 18s 128ms/step - loss: 0.6930 - accuracy: 0.5153 - val_loss: 0.6935 - val_accuracy: 0.4915
Epoch 2/50
14/14 [==============================] - 1s 40ms/step - loss: 0.6905 - accuracy: 0.5506 - val_loss: 0.6939 - val_accuracy: 0.4915
Epoch 3/50
14/14 [==============================] - 1s 40ms/step - loss: 0.6895 - accuracy: 0.5506 - val_loss: 0.6958 - val_accuracy: 0.4915
Epoch 4/50
14/14 [==============================] - 1s 39ms/step - loss: 0.6873 - accuracy: 0.5506 - val_loss: 0.6947 - val_accuracy: 0.4915
Epoch 5/50
14/14 [==============================] - 1s 40ms/step - loss: 0.6862 - accuracy: 0.5506 - val_loss: 0.6976 - val_accuracy: 0.4915
Epoch 6/50
14/14 [==============================] - 1s 40ms/step - loss: 0.6850 - accuracy: 0.5506 - val_loss: 0.7015 - val_accuracy: 0.4915
Epoch 7/50
14/14 [==============================] - 1s 40ms/step - loss: 0.6849 - accuracy: 0.5506 - val_loss: 0.6982 - val_accuracy: 0.4915
Epoch 8/50


In [ ]:
download_transfer = drive.CreateFile({'id':'1NTcd09yqjCh5a3kkWqiCGcclv8Nwo63q'})
download_transfer.GetContentFile('archive.zip')

In [ ]:
!unzip /content/archive.zip -d /content/transfer

Archive:  /content/archive.zip
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132320.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132449.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132516.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132540.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132608.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132636.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132704.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132740.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132814.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/Anota

In [ ]:
img_dir_transfer = "/content/transfer/New_Data_CoV2/Covid"
img_dirnon_transfer = "/content/transfer/New_Data_CoV2/Healthy"

healthy_patient_dirs = glob.glob("/content/transfer/New_Data_CoV2/Covid")
covid_patient_dirs = glob.glob("/content/transfer/New_Data_CoV2/Healthy")

# read in on an image-by-image basis
healthy_patient_imgs = glob.glob("/content/transfer/New_Data_CoV2/Healthy/*/*.png")
covid_patient_imgs = glob.glob("/content/transfer/New_Data_CoV2/Covid/*/*.png")

img_data =[]
img_label = []

for img in healthy_patient_imgs:
    img = cv2.imread(img)
    old_size = img.shape[:2]
    ratio = float(desired_size) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    img = cv2.resize(img, (new_size[1], new_size[0]))
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)
    color = [0, 0, 0]
    imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    normresize = (imgreszie* 1.0)/np.max(imgreszie)
    img_data.append(normresize)
    img_label.append([0])

for img in covid_patient_imgs:
    img = cv2.imread(img)
    old_size = img.shape[:2]
    ratio = float(desired_size) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    img = cv2.resize(img, (new_size[1], new_size[0]))
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)
    color = [0, 0, 0]
    imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    normresize = (imgreszie* 1.0)/np.max(imgreszie)
    img_data.append(normresize)
    img_label.append([1])

In [ ]:
img_data = np.array(img_data)
img_label = to_categorical(img_label)

In [ ]:
print(np.shape(img_data))
print(np.shape(img_label))

(2924, 100, 100, 3)
(2924, 2)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(img_data, img_label, test_size=0.40, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.40, random_state=1)

In [ ]:
checkpoint_filepath_1 = "/content/checkpoint_paper.hdf5"

In [ ]:
model.load_weights(checkpoint_filepath_1)


inputs6 = Input(shape = (desired_size, desired_size, 3))
add16 = (Conv2D(96, (3, 3), activation='relu', padding='valid'))(inputs6)
max16 = (MaxPooling2D((2, 2)))(add16)
add26 = (Conv2D(256, (5, 5), activation='relu', strides=1, padding='same'))(max16)
max26 = (MaxPooling2D((2, 2)))(add26)
add36 = (Conv2D(384, (3, 3), activation='relu', strides=1, padding='same'))(max26)
add46 = (Conv2D(384, (3, 3), activation='relu', strides=1, padding='same'))(add36)
add56 = (Conv2D(256, (3, 3), activation='relu', strides=1, padding='same'))(add46)
max36 = (MaxPooling2D((2, 2)))(add56)

addfinal = (Conv2D(128,(3,3),activation ='relu',padding='valid'))(max36)
finalmax = (MaxPooling2D((2,2)))(addfinal)
flattenmax = Flatten()(finalmax)
dense1 = (Dense(256, activation='relu'))(flattenmax)
dense2 = (Dense(512, activation='relu'))(dense1)
output = (Dense(2, activation='softmax'))(dense2)
model = Model(inputs = [inputs6],outputs = output)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 98, 98, 96)        2688      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 49, 49, 96)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 49, 49, 256)       614656    
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 24, 24, 256)       0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 24, 24, 384)       885120    
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 24, 24, 384)       1327

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = (true_positives * 1.0) / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = (true_positives * 1.0) / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# complie
optimizer = tf.optimizers.SGD(learning_rate=0.05)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])

In [ ]:
checkpoint_filepath_1 = '/content/checkpoint_transfer_paper.hdf5'
model_checkpoint_callback = tf.callbacks.ModelCheckpoint(filepath=checkpoint_filepath_1,save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, batch_size=32, epochs=50,callbacks=[model_checkpoint_callback],verbose=True,validation_data=(X_val,y_val))




Epoch 1/50
33/33 [==============================] - 5s 108ms/step - loss: 0.6020 - accuracy: 0.7433 - f1_m: 0.7430 - precision_m: 0.7430 - recall_m: 0.7430 - val_loss: 0.5715 - val_accuracy: 0.7393 - val_f1_m: 0.7388 - val_precision_m: 0.7388 - val_recall_m: 0.7388
Epoch 2/50
33/33 [==============================] - 1s 45ms/step - loss: 0.5587 - accuracy: 0.7433 - f1_m: 0.7432 - precision_m: 0.7432 - recall_m: 0.7432 - val_loss: 0.5509 - val_accuracy: 0.7393 - val_f1_m: 0.7388 - val_precision_m: 0.7388 - val_recall_m: 0.7388
Epoch 3/50
33/33 [==============================] - 2s 46ms/step - loss: 0.5676 - accuracy: 0.7433 - f1_m: 0.7435 - precision_m: 0.7435 - recall_m: 0.7435 - val_loss: 0.5418 - val_accuracy: 0.7393 - val_f1_m: 0.7388 - val_precision_m: 0.7388 - val_recall_m: 0.7388
Epoch 4/50
33/33 [==============================] - 2s 46ms/step - loss: 0.5419 - accuracy: 0.7433 - f1_m: 0.7438 - precision_m: 0.7438 - recall_m: 0.7438 - val_loss: 0.5615 - val_accuracy: 0.7393 - val_f

In [ ]:
model.load_weights(checkpoint_filepath_1)

loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=True)
print(f'Test loss: {loss:.3}')
print(f'Test accuracy: {accuracy:.3}')
print(f'Test f1_score: {f1_score:.3}')
print(f'Test precision: {precision:.3}')
print(f'Test recall: {recall:.3}')

37/37 [==============================] - 1s 28ms/step - loss: 0.2245 - accuracy: 0.9068 - f1_m: 0.9053 - precision_m: 0.9053 - recall_m: 0.9053
Test loss: 0.225
Test accuracy: 0.907
Test f1_score: 0.905
Test precision: 0.905
Test recall: 0.905
